# Detecting Issues in Regression Datasets (Erroneous Values in Numerical Data) 

This tutorial demonstrates how to use Cleanlab Studio's [Python API](/guide/quickstart/api/) to analyze a regression dataset.  In regression, the dataset labels correspond to a target (aka. outcome/dependent) variable and take continuous/numeric values (i.e. price, income, age) rather than discrete categories (for a similar tutorial on data with categorical labels, check out the [Tabular Quickstart](/tutorials/tabular_data_quickstart/)).

![Visualizing errors in a regression dataset](../assets/regression-tutorial/thumbnail.png)

Cleanlab Studio supports both text and tabular regression datasets. In this tutorial, we'll look at a tabular dataset where labels are students' final grades in a course (which may have data entry errors). This tutorial can be generally used to detect erroneous values in *any* numeric column of a dataset. 

Note: regression is currently only supported in the Python API in the generally-available version of Cleanlab Studio. If you require an interactive interface to improve your dataset, please [contact us](mailto:sales@cleanlab.ai) to discuss your use case.

## Install and import dependencies

In [ ]:
%pip install cleanlab-studio

In [1]:
import numpy as np
import pandas as pd
from cleanlab_studio import Studio

from IPython.display import display
pd.set_option("display.max_colwidth", None)

## Prepare and Upload Dataset

Our dataset for this tutorial is a collection of student grades and other information about each student. Suppose we are interested in detecting incorrect values (data entry errors) in students' `final_score`, which ranges from 0 to 100. In machine learning terminology, this can be considered a **regression** dataset, where the `final_score` column is the **label** for each row (student) in the table. 

### Load Dataset into Pandas DataFrame

We'll load the dataset into a Pandas DataFrame from a CSV file hosted in S3. The CSV file contains the following columns:

```
stud_ID,exam_1,exam_2,exam_3,notes,final_score
0,72,81,80,,73.3
1,89,62,93,,83.8
2,80,76,96,missed class frequently -10,78.6
<id of student>,<grade on exam 1>,<grade on exam 2>,<grade on exam 3>,<optional notes>,<overall letter grade for student>
...
```

You can similarly format any other tabular or text dataset and run the rest of this tutorial. Details on how to format your dataset can be found in [this guide](/guide/concepts/datasets/), which also outlines other format options. Cleanlab Studio works out-of-the-box for messy tabular datasets with arbitrary numeric/string columns that may contain missing values.

In [11]:
dataset_url = "https://cleanlab-public.s3.amazonaws.com/StudioDemoDatasets/student_grades_regression.csv"
df = pd.read_csv(dataset_url)
display(df.head(3))

,stud_ID,exam_1,exam_2,exam_3,notes,final_score
0,0,72,81,80,NaN,73.3
1,1,89,62,93,NaN,83.8
2,2,97,0,94,NaN,73.5


### Upload Dataset to Cleanlab Studio

Use your API key to instantiate a `Studio` object, which can be used to analyze your dataset.

In [2]:
# you can find your API key by going to app.cleanlab.ai/upload, 
# clicking "Upload via Python API", and copying the API key there
API_KEY = "<insert your API key>"

# authenticate with your API key
studio = Studio(API_KEY)

Next load the dataset into Cleanlab Studio (more details/options can be found in [this guide](/guide/quickstart/api/#uploading-a-dataset)). This may take a while for big datasets.

In [65]:
dataset_id = studio.upload_dataset(df, dataset_name="Student Grades (regression)", id_column="stud_ID")

Uploading dataset...: 100%|██████████|
Generating schema...: 100%|██████████|
Ingesting Dataset...: 100%|██████████|


## Launch a Project

Let's now create a project using this dataset. A Cleanlab Studio project will automatically train ML models to provide AI-based analysis of your dataset.

Here, we explicitly specify the column containing the class labels for each data point (`final_score` column in this dataset).

In [ ]:
project_id = studio.create_project(
    dataset_id=dataset_id,
    project_name="Student Grades (regression) Project",
    modality="tabular",
    task_type="regression",
    model_type="regular",
    label_column="final_score",
)
print(f"Project successfully created and training has begun! project_id: {project_id}")

Above, we specified `modality="tabular"` because this tutorial uses a tabular dataset; you can specify `modality="text"` if you're using a text dataset. See the documentation for [`create_project`](/reference/python/studio/#method-create_project) for the full set of options.

Once the project has been launched successfully and you see your `project_id`, feel free to close this notebook. It will take some time for Cleanlab’s AI to train on your data and analyze it. Come back after training is complete (you will receive an email) and continue with the notebook to review your results.

You should only execute the above cell once per dataset. After launching the project, you can poll for its status to programmatically wait until the results are ready for review. Each project creates a [cleanset](/guide/concepts/cleanset/), an improved version of your original dataset that contains additional metadata for helping you clean up the data. The next code cell simply waits until this *cleanset* has been created.

**Warning!** For big datasets, this next cell may take a long time to execute while Cleanlab's AI model is training. If your notebook has timed out during this process, you can resume work by re-running the below cell (which should return the `cleanset_id` instantly if the project has completed training). Do not re-run the above cell and create a new project.

In [3]:
cleanset_id = studio.get_latest_cleanset_id(project_id)
print(f"cleanset_id: {cleanset_id}")
project_status = studio.wait_until_cleanset_ready(cleanset_id)

cleanset_id: 52406e0b8af349e5962661bd95d67e56


Cleanset Progress: \ Step 18/18, Ready for review!


Once the above cell completes execution, your project results are ready for review!

## Download Cleanlab columns

We can fetch the [Cleanlab columns](/guide/concepts/cleanlab_columns/) that contain the metadata of this *cleanset* using its `cleanset_id`. These columns have the same length as your original dataset and provide metadata about each individual data point, like what types of issues it exhibits and how severe these issues are.

If at any point you want to re-run the remaining parts of this notebook (without creating another project), simply call `studio.download_cleanlab_columns(cleanset_id)` with the `cleanset_id` printed from the previous cell.

In [4]:
cleanlab_columns_df = studio.download_cleanlab_columns(cleanset_id)
cleanlab_columns_df.head()

,stud_ID,corrected_label,is_label_issue,label_issue_score,suggested_label,is_initially_unlabeled
0,0,NaN,False,0.821947,NaN,False
1,1,NaN,False,0.654694,NaN,False
2,2,NaN,True,0.979907,63.541901,False
3,3,NaN,False,0.645577,NaN,False
4,4,NaN,False,0.696790,NaN,False


## Review detected data issues 

**Optional: Initialize visualization helper functions**

We define some rule-based coloring functions to better visualize the numeric and string columns in this dataset.
Coloring helps us clearly understand data entry errors for this particular tutorial dataset, but is optional for your own datasets.


In [ ]:

from typing import Callable, Dict, Optional
from IPython.core.display import HTML
import matplotlib.colors

def color_calc(value: float):
    """
    Calculate color based on the given value. Intended for the range of 0 to 100. No error checking is done.
    
    Parameters:
    value (int or float): Value based on which the color is determined.
    
    Returns:
    str: Hexadecimal color code.
    """
    if value <= 50:
        r = 1.0
        g = (value / 50) * 194.0 / 255.0 + 61.0 / 255.0
        b = 61.0 / 255.0
    else:
        r = (100 - value) / 50
        g = 1.0
        b = 0.0
    hex_color = matplotlib.colors.to_hex((r, g, b))
    return hex_color

def grade_to_color(value: float):
    """
    Format numerical grade value with background color. Intended for the range of 0 to 100. No error checking is done.
    
    Parameters:
    value (int or float): Numerical grade value.
    
    Returns:
    str: HTML div string with background color based on grade value.
    """
    hex_color = color_calc(value)
    return f'<div style="background-color: {hex_color}; text-align: center;">{value}</div>'

def highlight_notes(note: str):
    """
    Format notes with background color based on keywords. Notes are returned as is if no keywords are found.
    
    Parameters:
    note (str): Text of notes.
    
    Returns:
    str: HTML div string with background color based on keywords found in notes.
    """
    if 'missed' in note:
        value = 40
    elif 'cheated' in note:
        value = 0
    elif 'great participation' in note:
        value = 100
    else:
        return note # default (no color)

    hex_color = color_calc(value)
    return f'<div style="background-color: {hex_color};">{note}</div>'

_TUTORIAL_FORMATTERS = {
    'exam_1': grade_to_color,
    'exam_2': grade_to_color,
    'exam_3': grade_to_color,
    'given_label': grade_to_color,
    'suggested_label': grade_to_color,
    'notes': highlight_notes
}
    

def display(df, formatters: Optional[Dict[str, Callable]] = None):
    """
    Display DataFrame with formatted columns.
    
    Parameters:
    df (pd.DataFrame): DataFrame to display.
    
    Returns:
    IPython.core.display.HTML: HTML representation of formatted DataFrame.
    """
    if formatters is None:
        formatters = {}
    return HTML(df.to_html(escape=False, formatters=formatters, index=False))


disable_pretty_print = False  # set to True to disable pretty printing when displaying DataFrames

optional_df_display_formatters = None if disable_pretty_print else _TUTORIAL_FORMATTERS

Details about all of the returned Cleanlab columns and their meanings can be found in [this guide](/guide/concepts/cleanlab_columns/). In this tutorial, we focus on columns related to label issues:
- **is_label_issue** indicates that the original value in the column you chose as the class label appears incorrect for a particular row (perhaps due to data entry error, or accidental mislabeling). For such data, consider correcting this value to the `suggested_label` value if it seems more appropriate.
- **label_issue_score** represents the severity of the label issue in each row (on a scale of 0-1 with 1 indicating the most severe instances of the issue).

Let's take a look at the label quality in our dataset. We first create a `given_label` column in our DataFrame to clearly indicate the class label originally assigned to each data point in this dataset.


In [36]:
# Combine the dataset with the cleanlab columns
combined_dataset_df = df.merge(cleanlab_columns_df, on="stud_ID")

# Set a "given_label" column to the original label
combined_dataset_df.rename(columns={"final_score": "given_label"}, inplace=True)

# Store the column names of the dataset for visualization
DATASET_COLUMNS = df.columns.drop("final_score").tolist()

To see which data points are estimated to be mislabeled (i.e. have potentially erroneous values in the class label column), we filter by `is_label_issue`. We sort by `label_issue_score` to see which of these data points are *most likely* mislabeled.

In [52]:
samples_ranked_by_label_issue_score = combined_dataset_df.sort_values("label_issue_score", ascending=False)

columns_to_display = DATASET_COLUMNS + ["label_issue_score", "is_label_issue", "given_label", "suggested_label"]
display(samples_ranked_by_label_issue_score.head(5)[columns_to_display], formatters=optional_df_display_formatters)

stud_ID,exam_1,exam_2,exam_3,notes,label_issue_score,is_label_issue,given_label,suggested_label
56,75,83,69,NaN,1.0,True,8.9,74.425201416
318,41,88,98,missed class frequently -10,1.0,True,0.0,71.7026596069
305,97,0,90,NaN,1.0,True,19.1,64.9820404053
560,95,0,72,NaN,1.0,True,0.0,50.9727096558
160,97,86,54,missed homework frequently -10,1.0,True,0.0,60.8798141479


For example, looking at the first row above: the student recieved grades of 75, 83, and 69 on their exams, but ended up with a final grade of 8.9/100, which is certainly a data entry error!

## Improve the dataset based on the detected issues

Since the results of this analysis appear reasonable, let's use the Cleanlab columns to improve the quality of our dataset. For your own datasets, which actions you should take to remedy the detected issues will depend on what you are using the data for. No single action is going to be the best choice across all datasets, so we caution against blindly copying the actions we perform below. 

For data marked as `label_issue`, we create a new `corrected_label` column, which will be the given label for data without detected label issues, and the `suggested_label` for data with detected label issues.

In [77]:
corrected_label = np.where(combined_dataset_df["is_label_issue"],
                           combined_dataset_df["suggested_label"],
                           combined_dataset_df["given_label"])

Finally, let's actually make a new version of our dataset with these changes.

We craft a new DataFrame from the original, applying corrections and exclusions, and then use this DataFrame to save the new dataset in a separate CSV file. The new dataset is a CSV file that looks just like our original dataset --- you can use it as a plug-in replacement to get more reliable results in your ML and Analytics pipelines, without any change in your existing modeling code.

In [ ]:
new_dataset_filename = "improved_dataset.csv"

In [ ]:
# Fetch the original dataset
fixed_dataset = combined_dataset_df[DATASET_COLUMNS].copy()

# Add the corrected label column 
fixed_dataset["final_score"] = corrected_label

# Save improved dataset to new CSV file
fixed_dataset.to_csv(new_dataset_filename, index=False)
print(f"Adjusted dataset saved to {new_dataset_filename}")